In [1]:
import os
import yaml
import json
import pandas as pd
from pathlib import Path

In [2]:
# parser.add_argument('mode', type=str,
#     choices=['make-ome', 'segment-ome', 'generate-spatial-features', 'show-channels'],
#     help='Which task mip is to execute.')

# # #################
# # ## segment-ome ##
# # #################
# parser.add_argument('--input-tif', type=str,
#     help='ome.tiff file to segment')

# parser.add_argument('--output-prefix', type=str, default='output',
#     help='Output prefix to use when writing cell segmentation files. Two files will be written: *_cell_segmentation.tif and *_nuclei_segmentation.tif. Default is "output". For example, if --output-prefix is "path/to/out/directory/sample" then the two output files will be named path/to/out/directory/sample_cell_segmentation.tif and path/to/out/directory/sample_nuclei_segmentation.tif.')

# parser.add_argument('--split-size', type=int, default=25000,
#     help='If image width or height is larger than --split-size, then image will be split into multiple pieces for segmentation and stitched back together. Decrease if running into memory issues.')

# parser.add_argument('--nuclei-channels', type=str,
#     help='List of nuclei markers to use during segmentation. Must be the following format: "MARKER_NAME,MARKER_NAME,....". Default is "DAPI".')

# parser.add_argument('--membrane-channels', type=str,
#     help='List of markers to use during membrane segmentation. Must be the following format: "MARKER_NAME,MARKER_NAME,....". Default is "Pan-Cytokeratin,E-cadherin,CD45,CD8,CD3e,Vimentin,SMA,CD31,CD20". Note that image marker names are automatically converted using mip.utils.R_CHANNEL_MAPPING')


In [3]:
cwl_dir = '/diskmnt/Projects/Users/estorrs/multiplex-imaging-pipeline/cwl'
Path(cwl_dir).mkdir(parents=True, exist_ok=True)

In [4]:
template = {
    'membrane_channels': 'a_string',
    'nuclei_channels': 'a_string',
    'split_size': 'a_int',
    'output_prefix': 'a_string',
    'input_tif': {
        'class': 'File',
        'path': 'a/file/path',
    },
}

In [5]:
yaml.safe_dump(template, open(os.path.join(cwl_dir, 'template.segmentation.yaml'), 'w'))

In [10]:
cwl = {
    'class': 'CommandLineTool',
    'cwlVersion': 'v1.0',
    '$namespaces': {'sbg': 'https://www.sevenbridges.com/'},
    'id': 'mesmer_segmentation',
    'label': 'mesmer_segmentation',
    'baseCommand': ['mip'],
    'inputs': [
        {
            'id': 'input_tif',
            'type': 'File',
            'inputBinding': {
                'position': '0',
                'prefix': '--input-tif',
            }
        },
        {
            'id': 'output_prefix',
            'type': 'string?',
            'default': 'output',
            'inputBinding': {
                'prefix': '--output-prefix',
                'position': '0'
            }
        },
        {
            'id': 'nuclei_channels',
            'type': 'string?',
            'default': 'DAPI',
            'inputBinding': {
                'prefix': '--nuclei-channels',
                'position': '0'
            }
        },
        {
            'id': 'membrane_channels',
            'type': 'string?',
            'default': 'Pan-Cytokeratin,E-cadherin,CD45,CD8,CD3e,Vimentin,SMA,CD31,C20',
            'inputBinding': {
                'prefix': '--membrane-channels',
                'position': '0'
            }
        },
        {
            'id': 'split_size',
            'type': 'int?',
            'default': 25000,
            'inputBinding': {
                'prefix': '--split-size',
                'position': '0'
            }
        },
        
        # needs path to be set so it works on compute1
        {
            'id': 'environ_PATH',
            'type': 'string?',
            'default': '/miniconda/envs/mip/bin:$PATH'
        },
    ],
    'arguments': [
        {
            'position': 1,
            'valueFrom': 'segment-ome'
        },
    ],
    'outputs': [
        {
            'id': 'labeled_cells',
            'type': 'File',
            'outputBinding': {'glob': '*_cell_segmentation.tif'}
        },
        {
            'id': 'labeled_nuclei',
            'type': 'File',
            'outputBinding': {'glob': '*_nuclei_segmentation.tif'}
        },
    ],
    'requirements': [
        {
            'class': 'DockerRequirement',
            'dockerPull': 'estorrs/multiplex-imaging-pipeline:0.0.1'
        },
        {
            'class': 'ResourceRequirement',
            'ramMin': 100000
        },
        {
            'class': 'EnvVarRequirement',
            'envDef': {
                'PATH': '$(inputs.environ_PATH)',
            }
        }
    ]
}

In [11]:
yaml.safe_dump(cwl, open(os.path.join(cwl_dir, 'segmentation.cwl'), 'w'))